# LangChain: Models, Prompts and Output Parsers

In [ ]:
!pip install litellm

In [ ]:
# Import necessary libraries
import os
import litellm
from litellm import completion

# Set the OpenAI API key directly from environment variables.
# This API key is used for authentication with the litellm service.
litellm.openai_key = ''
litellm.api_base = ''

## Chat API

Let's start with a direct API.

In [ ]:
# Helper function to interact with a model
# We use the qwen 2.5 model and send a prompt in the format required for chat completions.

def get_completion(prompt, model="openai/qwen2.5", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = litellm.completion(
        model=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
# Test the function with a simple arithmetic question
get_completion("What is 1+1?")

In [ ]:
# Example customer email and style specification for response generation
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a calm and respectful tone
"""

In [ ]:
# Format a prompt to translate the customer email into a specified style
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [ ]:
# Generate a response based on the prompt
response = get_completion(prompt)

In [ ]:
response

## Chat API : LangChain

Let's try how we can do the same using LangChain.

### Installation

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_openai

### Model

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
# Initialize the chat model with specified API key, model, and base URL.
# Temperature is set to 0.0 to limit randomness in output.
chat = ChatOpenAI(model="qwen2.5",
                  api_key="",
                  base_url='',
                  temperature=0.0)

### Prompt template

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Create a template from which prompts can be formatted dynamically
prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
# Define customer message style and content to format the prompt
customer_style = """American English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(customer_response.content)

In [ ]:
# Create a response in a different tone and format it as pirate English
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [ ]:
# Example structured JSON output for parsed responses
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [ ]:
# Sample customer review for parsing
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
# chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

In [ ]:
type(response.content)

In [ ]:
# You will get an error by running this line of code
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
# Define schemas for each expected key in the output
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
# Combine schemas into a structured output parser
response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# Define format instructions for the prompt
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
# Create a new template including format instructions
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)
print(messages[0].content)

In [ ]:
response = chat(messages)
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')